In [96]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
#
import importlib
import utilities.lstm_utils as lstm_utils
import utilities.mpt_utils as mpt_utils
import utilities.variables as variables

In [97]:
# set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Data

In [98]:
df = pd.read_csv('../../../data/df_monthly_prices_complete_euro.csv', index_col='Date')
df_pct = pd.read_csv('../../../data/df_monthly_returns_complete.csv', index_col='Date')
df_overview = pd.read_csv('../../../data/df_overview.csv', index_col=0)

## LSTM Model

In [99]:
# Define 4.3. LSTM model
class LSTM_Uni_Model(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=1, output_size=1, learning_rate=0.001, dropout=0.2): # , hidden_size=128
        super(LSTM_Uni_Model, self).__init__()
        self.hidden_size = hidden_size
        # init LSTM
        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            # num_layers=num_layers,
                            batch_first=True)

        # FC layer for final prediction
        self.fc_final = nn.Linear(hidden_size, 12)

    def forward(self, ts_batch): # ts_batch (64, 1653, 10), static_data (64, 1653, 44)
        # Time-Series Data
        # Reshape dynamic data for LSTM (requires time-step as 2nd dimension)
        batch_size, num_stocks, sequence_length = ts_batch.shape[0], ts_batch.shape[1], ts_batch.shape[2]
        ts_batch_reshaped = ts_batch.view(batch_size * num_stocks, sequence_length)
        #
        ts_output_1, (hidden, cell)  = self.lstm(ts_batch_reshaped) # ts_batch_reshaped
        #
        ts_output = ts_output_1.view(batch_size, num_stocks, self.hidden_size)

        return self.fc_final(ts_output)#.squeeze(-1) # ts_output_2

We use a 12 month lookback for the sequential data to predict the upcoming 12 months.

After that, based on the currently predicted time-horizon, we get the respective sub-range, 
be it 1-month, 6-month or 12-month ahead.

In [100]:
# Set sequence length (12 months)
in_seq_length = 12
out_seq_length = 12
#
out_seq_length_1m = 1
out_seq_length_6m = 6
out_seq_length_12m = 12

### LSTM Univariate

In [101]:
df_to_evaluate = df_pct - 1

#### Train-Test Splits

Split the data into training and testing sets

In [102]:
importlib.reload(lstm_utils)
importlib.reload(variables)

# Set sequence length (e.g., 12 time points)
X_train, X_test, y_train, y_test = lstm_utils.split_train_test(df_to_evaluate, [], 
                                                               in_seq_length=in_seq_length, 
                                                               out_seq_length=out_seq_length, 
                                                               validation_months=(variables.LSTM_TEST_YEARS_NR * 12))

# Check the shapes of the training and test data
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: torch.Size([215, 1332, 12])
Shape of y_train: torch.Size([215, 1332, 12])
Shape of X_test: torch.Size([60, 1332, 12])
Shape of y_test: torch.Size([60, 1332, 12])


### Model Training

In [103]:
X_test.shape

torch.Size([60, 1332, 12])

In [104]:
y_test.shape

torch.Size([60, 1332, 12])

In [105]:
# Model, Loss, Optimizer
model = LSTM_Uni_Model(input_size=in_seq_length, output_size=out_seq_length).to(device)
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)

importlib.reload(lstm_utils)
#
model, y_train_pred, y_test_pred = lstm_utils.lstm_train_validate(model, optimizer, X_train, X_test, y_train, y_test)

Epoch 1/100, Loss: 0.0140, Train RMSE: 0.1188, Test RMSE: 0.1273. 
Epoch 2/100, Loss: 0.0131, Train RMSE: 0.1145, Test RMSE: 0.1271. 
Epoch 3/100, Loss: 0.0129, Train RMSE: 0.1137, Test RMSE: 0.1267. 
Epoch 4/100, Loss: 0.0129, Train RMSE: 0.1136, Test RMSE: 0.1267. 
Epoch 5/100, Loss: 0.0128, Train RMSE: 0.1134, Test RMSE: 0.1267. 
Epoch 6/100, Loss: 0.0128, Train RMSE: 0.1134, Test RMSE: 0.1267. 
Epoch 7/100, Loss: 0.0128, Train RMSE: 0.1133, Test RMSE: 0.1267. 
Epoch 8/100, Loss: 0.0128, Train RMSE: 0.1133, Test RMSE: 0.1267. 
Epoch 9/100, Loss: 0.0128, Train RMSE: 0.1133, Test RMSE: 0.1267. 
Epoch 10/100, Loss: 0.0128, Train RMSE: 0.1133, Test RMSE: 0.1268. 
Epoch 11/100, Loss: 0.0128, Train RMSE: 0.1133, Test RMSE: 0.1268. 
Epoch 12/100, Loss: 0.0128, Train RMSE: 0.1133, Test RMSE: 0.1268. 
Epoch 13/100, Loss: 0.0128, Train RMSE: 0.1132, Test RMSE: 0.1268. 
Epoch 14/100, Loss: 0.0128, Train RMSE: 0.1132, Test RMSE: 0.1268. 
Epoch 15/100, Loss: 0.0128, Train RMSE: 0.1132, Test RMSE

### LSTM Univariate - 1 Month

Get the known data (train data).
After that, get the first predicted month, or the first predicted sequence of test data

In [106]:
# 1 month
df_train = X_train[:, :, -1].clone()
y_test_pred_1m = y_test_pred[0,:,:].T[0:0] # y_test_pred[:,:,0][0] 
y_test_pred_1m

tensor([], size=(0, 1332))

In [107]:
df_forecast_1m = pd.DataFrame(df_train.clone())
df_forecast_1m = pd.concat([df_forecast_1m, pd.DataFrame(y_test_pred_1m)], ignore_index=True)
# Assign back columns and indices to make human understandable
df_forecast_1m.columns = df.columns
df_forecast_1m.index = pd.to_datetime(df_pct[(in_seq_length - 1) : len(df_forecast_1m) + (in_seq_length - 1)].index)
#
df_forecast_1m = df_forecast_1m.tail(variables.LSTM_TEST_YEARS_NR * 12) 
#
df_forecast_1m.tail(3)

,GME,2124.T,2491.T,2471.T,3046.T,PAT.DE,CROX,AOF.DE,SFQ.DE,DAN,...,KREF,HLN.L,DBX,BNL,CBL,KVUE,PSTL,NTST,BLCO,NBS.L
Date,,,,,,,,,,,,,,,,,,,,,
2018-06-01,0.10,0.10,0.25,-0.24,0.04,-0.07,-0.01,0.10,-0.18,-0.09,...,-0.03,0.0,0.08,0.0,0.0,0.0,0.02,0.02,0.0,0.0
2018-07-01,0.02,0.00,0.04,0.10,0.00,0.19,0.03,-0.01,0.10,0.06,...,0.07,0.0,-0.17,0.0,0.0,0.0,0.00,0.00,0.0,0.0
2018-08-01,-0.08,0.01,0.07,0.13,-0.09,-0.07,0.14,-0.07,-0.06,-0.08,...,0.02,0.0,0.00,0.0,0.0,0.0,-0.01,0.00,0.0,0.0


In [109]:
importlib.reload(lstm_utils)
importlib.reload(mpt_utils)

weights_1m, mu_1m, S_1m, allocations_1m, weights_all_1m = mpt_utils.portfolio_and_plot(df_forecast_1m, df, plot_threshold=0.018)

Expected annual return: 81.8%
Annual volatility: 6.9%
Sharpe Ratio: 11.54
-- Allocation --
{'ETSY': 1, '2491.T': 6, 'TPE.DE': 2, 'CVNA': 1, '3697.T': 1, 'PAR': 1, 'FCN': 1, 'FNKO': 4, 'NTNX': 1, 'RUN': 4, '6951.T': 1, '7518.T': 2, 'ABDP.L': 1, 'BOOT': 1, 'LULU': 1, 'HSII': 1, 'ADUS': 1, 'NSP': 1, 'IIPR': 1, '2471.T': 11, '6814.T': 3, 'WSC': 1, 'HAE': 1, 'MEDP': 1, 'AEO': 1, '4568.T': 1, 'QLYS': 1, '6866.T': 1, 'NSSC': 1, '9697.T': 1, '6200.T': 3, '4751.T': 5, 'CATO': 4, '2475.T': 2, 'MOV': 1, 'BBSI': 1, '2429.T': 3, '2389.T': 2, 'EH': 2, '2154.T': 2, '3994.T': 1, 'SHYF': 2, 'COK.DE': 1, '7730.T': 1, 'FARO': 1, '4218.T': 2, '4641.T': 1, 'GES': 1, 'PRDO': 1, '2170.T': 7, '6914.T': 2, '4503.T': 1, 'TPR': 1, 'UAA': 3, '9413.T': 1, '7868.T': 5, 'NEO': 1, '6183.T': 1, '7990.T': 1, 'FTK.DE': 1, 'NVRI': 3, '6055.T': 1, '7296.T': 1, '3151.T': 2, '7476.T': 1, '3048.T': 2, '7575.T': 3, 'CNK': 1, '7613.T': 2, '7936.T': 2, 'PKE': 1, '7220.T': 1, '4301.T': 1, '8439.T': 1, '3738.T': 1, '7752.T': 2, '

/Users/herbishtini/anaconda3/envs/portfolio_optimisation/lib/python3.11/site-packages/pypfopt/discrete_allocation.py:27: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



#### Actual return rate

In [110]:
importlib.reload(mpt_utils)
#
mpt_utils.get_portfolio_real_return_rate(df_pct, df_forecast_1m, weights=weights_all_1m)

Portfolio real return rate:  0.82%


#### Overview table

In [111]:
importlib.reload(mpt_utils)
# Create overview
mpt_utils.generate_overview_table(allocations_1m, mu_1m, S_1m, df_pct)

,Share Count,Average Covariance,Average Returns,Return Last 12 Months,Return (Actual) Next 12 Months
ETSY,1,-0.004421,47.69%,-33.98%,-19.03%
2491.T,6,-0.024421,182.25%,-40.43%,-10.18%
TPE.DE,2,0.003474,183.13%,-14.53%,-17.65%
CVNA,1,-0.062842,27.62%,98.50%,211.25%
3697.T,1,0.013053,170.34%,15.27%,-56.94%
...,...,...,...,...,...
7893.T,1,0.019579,72.08%,27.21%,7.98%
TRTX,1,0.000421,14.39%,-25.34%,49.38%
AMCR,1,0.010947,-13.23%,-20.57%,26.52%
HBI,1,0.012737,11.15%,-37.03%,54.02%


### LSTM Univariate - 6 Months

Get the known data (train data)
After that, get the first 6 predicted months, or the first 6 predicted sequences of test data

In [112]:
df_train = X_train[:, :, -1].clone()
y_test_pred_6m =  y_test_pred[0,:,:].T[0:6] # y_test_pred[:,:,5][0:6]
y_test_pred_6m

tensor([[-2.2862e-02, -1.6169e-02, -1.6841e-02,  ..., -2.8487e-04,
          3.8905e-05, -1.6906e-03],
        [ 3.6470e-03, -7.3244e-03, -6.7286e-03,  ...,  2.1560e-03,
          2.2464e-03,  1.8057e-03],
        [-2.0139e-02, -1.6514e-02, -1.1265e-02,  ...,  2.6666e-03,
          2.8850e-03,  2.6790e-03],
        [ 4.6193e-02,  2.9297e-02,  2.0988e-02,  ...,  3.1837e-03,
          2.9743e-03,  2.6965e-03],
        [ 3.9210e-02,  2.1783e-02,  1.3951e-02,  ...,  2.3369e-03,
          2.4041e-03,  1.4492e-03],
        [-1.2721e-02, -6.1398e-03, -7.7279e-03,  ...,  7.2468e-03,
          7.1753e-03,  6.6094e-03]])

In [113]:
df_forecast_6m = pd.DataFrame(df_train.clone())
df_forecast_6m = pd.concat([df_forecast_6m, pd.DataFrame(y_test_pred_6m)], ignore_index=True)
# Assign back columns and indices to make human understandable
df_forecast_6m.columns = df.columns
df_forecast_6m.index = pd.to_datetime(df_pct[(in_seq_length - 1) : len(df_forecast_6m) + (in_seq_length - 1)].index)
#
df_forecast_6m = df_forecast_6m.tail(variables.LSTM_TEST_YEARS_NR * 12)
#
df_forecast_6m.tail(3)

,GME,2124.T,2491.T,2471.T,3046.T,PAT.DE,CROX,AOF.DE,SFQ.DE,DAN,...,KREF,HLN.L,DBX,BNL,CBL,KVUE,PSTL,NTST,BLCO,NBS.L
Date,,,,,,,,,,,,,,,,,,,,,
2018-12-01,0.046193,0.029297,0.020988,0.023813,0.019041,0.011086,0.007820,0.011091,0.010090,0.009012,...,0.002784,0.003136,0.005476,0.004420,0.003963,0.003874,0.003653,0.003184,0.002974,0.002696
2019-01-01,0.039210,0.021783,0.013951,0.003016,0.011832,0.009292,0.001084,0.006255,0.006795,0.004621,...,0.001943,0.001222,0.002200,0.002587,0.002434,0.001812,0.002270,0.002337,0.002404,0.001449
2019-02-01,-0.012721,-0.006140,-0.007728,0.001715,0.013797,0.017453,0.018494,0.015117,0.019044,0.020182,...,0.011926,0.010787,0.009322,0.008674,0.008118,0.007243,0.007662,0.007247,0.007175,0.006609


In [115]:
importlib.reload(lstm_utils)
importlib.reload(mpt_utils)

weights_6m, mu_6m, S_6m, allocations_6m, weights_all_6m = mpt_utils.portfolio_and_plot(df_forecast_6m, df, plot_threshold=0.008)

Expected annual return: 19.0%
Annual volatility: 4.1%
Sharpe Ratio: 4.15
-- Allocation --
{'FNKO': 5, 'CVNA': 2, 'PAR': 1, 'CON.DE': 1, 'MDB': 1, '2685.T': 4, 'CRUS': 1, 'VSEC': 1, 'STAA': 1, 'NBPE.L': 1, '2120.T': 50, 'NSSC': 1, '7575.T': 9, 'SHYF': 3, 'CEC.DE': 32, '8154.T': 1, '6268.T': 3, 'RMBS': 1, 'KMX': 1, 'OC': 1, 'MHK': 1, '8963.T': 1, 'CPT': 1, 'HD': 1, 'RGLD': 1, 'DAN': 4, 'MMSI': 1, 'UNM': 1, '1925.T': 1, 'AMD': 1, '7239.T': 3, 'BBSI': 1, '4218.T': 3, 'ZD': 1, '8897.T': 12, 'WHR': 1, 'LOPE': 1, '6745.T': 3, '6436.T': 1, 'CENT': 1, 'NSP': 1, 'KFY': 1, 'LCII': 1, 'LAD': 1, '6941.T': 2, '5988.T': 2, 'DLTR': 1, '3659.T': 1, 'MET': 1, '2429.T': 2, '9790.T': 1, 'ETSY': 1, 'IGT': 1, 'CCK': 1, 'DORM': 1, 'GEN': 2, '9842.T': 2, 'SVS.L': 1, 'MBG.DE': 1, '3104.T': 1, 'THRM': 1, 'MPWR': 1, 'SFQ.DE': 2, '2502.T': 1, '2331.T': 4, '3046.T': 1, '6856.T': 1, 'SCSC': 1, '7915.T': 2, 'TTK.DE': 2, '9766.T': 1, '8219.T': 3, '7988.T': 1, '5110.T': 2, 'NEO': 1, '4549.T': 2, 'AMKR': 1, '3402.T': 6

/Users/herbishtini/anaconda3/envs/portfolio_optimisation/lib/python3.11/site-packages/pypfopt/discrete_allocation.py:27: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



#### Actual return rates

In [116]:
importlib.reload(mpt_utils)
#
mpt_utils.get_portfolio_real_return_rate(df_pct, df_forecast_6m, weights=weights_all_6m)

Portfolio real return rate:  0.08%


#### Overview table

In [117]:
importlib.reload(mpt_utils)
# Create overview
mpt_utils.generate_overview_table(allocations_6m, mu_6m, S_6m, df_pct)

,Share Count,Average Covariance,Average Returns,Return Last 12 Months,Return (Actual) Next 12 Months
FNKO,5,-0.070129,17.77%,-62.69%,34.43%
CVNA,2,-0.072516,5.19%,98.50%,211.25%
PAR,1,-0.043097,-19.02%,1.20%,41.61%
CON.DE,1,0.024968,-20.02%,19.48%,-18.83%
MDB,1,-0.067097,10.84%,88.40%,-15.01%
...,...,...,...,...,...
TRTX,1,0.000516,-3.81%,-25.34%,49.38%
AGNC,1,0.003355,-14.27%,4.01%,28.11%
3608.T,1,0.021419,12.16%,81.27%,12.53%
BDN,2,-0.002839,21.89%,-34.15%,24.52%


### LSTM Univariate - 12 Months

Get the known data (train data)
After that, get the first 12 predicted months, or the first 12 predicted sequences of test data

In [118]:
df_train = X_train[:, :, -1].clone()
y_test_pred_12m = y_test_pred[0,:,:].T[0:12]
y_test_pred_12m

tensor([[-2.2862e-02, -1.6169e-02, -1.6841e-02,  ..., -2.8487e-04,
          3.8905e-05, -1.6906e-03],
        [ 3.6470e-03, -7.3244e-03, -6.7286e-03,  ...,  2.1560e-03,
          2.2464e-03,  1.8057e-03],
        [-2.0139e-02, -1.6514e-02, -1.1265e-02,  ...,  2.6666e-03,
          2.8850e-03,  2.6790e-03],
        ...,
        [-1.0484e-02, -5.9916e-03, -1.6203e-03,  ...,  8.6390e-03,
          8.6631e-03,  9.7183e-03],
        [ 5.1967e-02,  3.4324e-02,  1.8598e-02,  ...,  8.3613e-03,
          8.5316e-03,  8.4484e-03],
        [ 5.6362e-02,  3.6120e-02,  2.1328e-02,  ...,  9.2894e-03,
          9.3684e-03,  8.0125e-03]])

In [119]:
df_forecast_12m = pd.DataFrame(df_train.clone())
df_forecast_12m = pd.concat([df_forecast_12m, pd.DataFrame(y_test_pred_12m)], ignore_index=True)
# Assign back columns and indices to make human understandable
df_forecast_12m.columns = df.columns
df_forecast_12m.index = pd.to_datetime(df_pct[(in_seq_length - 1) : len(df_forecast_12m) + (in_seq_length - 1)].index)
#
df_forecast_12m.tail(3)

,GME,2124.T,2491.T,2471.T,3046.T,PAT.DE,CROX,AOF.DE,SFQ.DE,DAN,...,KREF,HLN.L,DBX,BNL,CBL,KVUE,PSTL,NTST,BLCO,NBS.L
Date,,,,,,,,,,,,,,,,,,,,,
2019-06-01,-0.010484,-0.005992,-0.001620,0.001425,0.005831,0.009403,0.004748,0.008407,0.014416,0.015395,...,0.011456,0.010821,0.008187,0.009303,0.009576,0.009728,0.009390,0.008639,0.008663,0.009718
2019-07-01,0.051967,0.034324,0.018598,0.016274,0.014241,0.012138,0.007054,0.006717,0.010763,0.011026,...,0.008136,0.008191,0.009931,0.008957,0.008626,0.008563,0.008250,0.008361,0.008532,0.008448
2019-08-01,0.056362,0.036120,0.021328,0.002568,0.012057,0.007178,0.007241,0.006356,0.005094,0.008412,...,0.004660,0.006193,0.009908,0.008834,0.008734,0.008356,0.009111,0.009289,0.009368,0.008013


In [121]:
importlib.reload(lstm_utils)
importlib.reload(mpt_utils)

weights_12m, mu_12m, S_12m, allocations_12m, weights_all_12m = mpt_utils.portfolio_and_plot(df_forecast_12m, df, plot_threshold=0.014)

Expected annual return: 12.7%
Annual volatility: 0.6%
Sharpe Ratio: 17.69
-- Allocation --
{'MDB': 1, 'ROKU': 6, 'MMSI': 4, 'ENPH': 3, '3046.T': 8, 'SHYF': 20, 'DDS': 1, 'TEP.L': 1, '2874.T': 28, '2120.T': 131, 'SNX': 2, 'AMG': 1, '9790.T': 9, 'SAIA': 1, '2170.T': 62, 'SONO': 17, '4218.T': 13, 'ETSY': 2, 'UAA': 23, 'ZD': 2, 'HD': 1, 'PAR': 4, 'CATO': 22, '7947.T': 6, '1925.T': 4, 'CGNX': 3, 'CCK': 1, 'UNM': 2, 'PRI': 1, '8154.T': 2, '4298.T': 13, '8876.T': 10, 'GEN': 6, '4293.T': 32, 'PAT.DE': 12, 'DE': 1, '2331.T': 16, 'VRNT': 5, 'AIV': 15, 'SNWS.L': 1, '7740.T': 6, '9107.T': 7, 'SAX.DE': 1, '7272.T': 9, 'DAVA': 1, '8012.T': 4, 'CON.DE': 1, '9104.T': 2, 'CRI': 1, '2685.T': 2, 'BXMT': 3, '9831.T': 18, 'CLMB': 1, 'MAC': 6, 'INN': 10, '7701.T': 2, '9470.T': 9, '1911.T': 1, 'STGW': 10, '8060.T': 1, 'SNBR': 2, 'ABM': 1, 'AVTR': 2, '9364.T': 1, 'HBI': 9, '8173.T': 2, 'UDR': 1, 'EH': 1, '7914.T': 1, 'CHEF': 1, '8818.T': 2, '7239.T': 2, '8159.T': 1, '8439.T': 2, 'MIR': 2}
-- Weights Percentag

/Users/herbishtini/anaconda3/envs/portfolio_optimisation/lib/python3.11/site-packages/pypfopt/discrete_allocation.py:27: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



#### Actual return rates

In [122]:
importlib.reload(mpt_utils)
#
mpt_utils.get_portfolio_real_return_rate(df_pct, df_forecast_12m, weights=weights_all_12m, months=12)

Portfolio real return rate:  0.22%


#### Overview Table

In [123]:
importlib.reload(mpt_utils)
# Create overview
mpt_utils.generate_overview_table(allocations_12m, mu_12m, S_12m, df_pct)

,Share Count,Average Covariance,Average Returns,Return Last 12 Months,Return (Actual) Next 12 Months
MDB,1,0.000000,11.91%,88.40%,-15.01%
ROKU,6,-0.000541,12.63%,8.15%,-8.01%
MMSI,4,0.000000,11.11%,0.48%,39.39%
ENPH,3,-0.001216,13.83%,-74.11%,-13.52%
3046.T,8,0.000000,20.76%,-4.69%,66.83%
...,...,...,...,...,...
8818.T,2,0.000000,6.22%,8.82%,17.35%
7239.T,2,0.000000,9.39%,55.39%,17.28%
8159.T,1,0.000000,10.02%,65.71%,-2.23%
8439.T,2,0.000000,7.15%,16.81%,17.89%
